# Biome Level statistics for model: GS2_MaxScaler


In [17]:
# import the libraries
import ee
import pandas as pd
import os
import numpy as np
import random
from random import sample
import itertools 
import geopandas as gpd
from sklearn.metrics import r2_score
from termcolor import colored # this is allocate colour and fonts type for the print title and text
from IPython.display import display, HTML

In [18]:
#set the working directory of local drive for Grid search result table loading
# os.getcwd()


In [19]:
# initialize the earth engine API
ee.Initialize()

## 1 Load the required composites and images

In [20]:
# load the basic maps that needed for the analysis
# load the carbon concentration map
carbonConcentration = ee.Image("users/leonidmoore/ForestBiomass/Biome_level_Wood_Carbon_Conentration_Map")
# load the root shoot ratio map
rootShootRatio = ee.Image("users/leonidmoore/ForestBiomass/Root_shoot_ratio_Map").unmask()
# load the two composites tha will be used in the analysis
compositeImage =ee.Image("users/leonidmoore/ForestBiomass/20200915_Forest_Biomass_Predictors_Image")
compositeImageNew = ee.Image("projects/crowtherlab/Composite/CrowtherLab_Composite_30ArcSec")
# load the biome layer 
biomeLayer = compositeImage.select("WWF_Biome")
# define a pixel area layer with unit km2
pixelAreaMap = ee.Image.pixelArea().divide(10000);
# define the boundary geography reference
unboundedGeo = ee.Geometry.Polygon([-180, 88, 0, 88, 180, 88, 180, -88, 0, -88, -180, -88], None, False)

## 2 Load the biomass density maps

In [21]:
# load the carbon density layers
potentialDensity = ee.Image("users/leonidmoore/ForestBiomass/GroundSourcedModel/EnsambledMaps/Predicted_GS2_MaxScaler_Potential_density_Ensambled_Mean_20230427").unmask()
presentDensity = ee.Image("users/leonidmoore/ForestBiomass/GroundSourcedModel/EnsambledMaps/Predicted_GS1_MaxScaler_Present_density_Ensambled_Mean_20230427").unmask()

# define the standard projection
stdProj = potentialDensity.projection()

## 3 Adjust the present and potential density maps

In [22]:
# load the present and potential forest cover
presentForestCover = compositeImage.select('PresentTreeCover').unmask() # uniform with potential in the  0-1 scale
potentialCoverAdjusted = ee.Image("users/leonidmoore/ForestBiomass/Bastin_et_al_2019_Potential_Forest_Cover_Adjusted").unmask().rename('PotentialForestCover')
# define the present and potential forest cover masks
presentMask= presentForestCover.gt(0)
potentialMask= potentialCoverAdjusted.gt(0)

# check the difference of the two density maps
potentialHigher = potentialDensity.multiply(pixelAreaMap).subtract(presentDensity.multiply(pixelAreaMap)).gte(0)
potentialLower = potentialDensity.multiply(pixelAreaMap).subtract(presentDensity.multiply(pixelAreaMap)).lt(0)
# replace the lower potential value by present biomass density value
agbPotentialDensity = presentDensity.multiply(potentialLower).add(potentialDensity.multiply(potentialHigher))
# add the root biomass to the AGB to get TGB
tgbPotentialDensity = agbPotentialDensity.multiply(rootShootRatio).add(agbPotentialDensity)
tgbPresentDensity = presentDensity.multiply(rootShootRatio).add(presentDensity)

## 4 Partioning the potential cover into different landuse types

In [23]:
# Load all the landuse type layers
croplandOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/cropland_Percent").rename('cropland').divide(100).reproject(crs=stdProj);
grazingOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/grazing_Percent").rename('grazing').divide(100).reproject(crs=stdProj);
pastureOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/pasture_Percent").rename('pasture').divide(100).reproject(crs=stdProj);
rangelandOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/rangeland_Percent").rename('rangeland').divide(100).reproject(crs=stdProj);
urbanOrg = compositeImage.select(['LandCoverClass_Urban_Builtup']).divide(100).unmask().reproject(crs=stdProj);
snowIceOrg = compositeImageNew.select(['ConsensusLandCoverClass_Snow_Ice']).divide(100).unmask().reproject(crs=stdProj);
openWaterOrg = compositeImageNew.select(['ConsensusLandCoverClass_Open_Water']).divide(100).unmask().reproject(crs=stdProj);
# define the total landcover types
sumCover = presentForestCover.add(pastureOrg).add(rangelandOrg).add(croplandOrg).add(urbanOrg).add(openWaterOrg).add(snowIceOrg);
oneSubtract = ee.Image(1).subtract(sumCover);
freeland = oneSubtract.multiply(oneSubtract.gte(0));
# get the scale ratio for pixels with sumCover larger than 1
scaleRatio = ee.Image(1).subtract(presentForestCover).divide(sumCover.subtract(presentForestCover)).multiply(oneSubtract.lt(0));
# get the ratio of these three disturbed maps
pasture = pastureOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(pastureOrg.multiply(oneSubtract.gte(0))).unmask();
rangeland = rangelandOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(rangelandOrg.multiply(oneSubtract.gte(0))).unmask();
cropland = croplandOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(croplandOrg.multiply(oneSubtract.gte(0))).unmask();
urban = urbanOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(urbanOrg.multiply(oneSubtract.gte(0))).unmask();
openWater = openWaterOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(openWaterOrg.multiply(oneSubtract.gte(0))).unmask();
snowIce = snowIceOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(snowIceOrg.multiply(oneSubtract.gte(0))).unmask();
sumTT = presentForestCover.add(pasture).add(rangeland).add(cropland).add(urban).add(freeland).add(openWater).add(snowIce).unmask();

effectivePotentialMask = freeland.add(rangeland).add(pasture).add(cropland).add(urban).gt(0);
# there are some pixels without any landcover survived but with open water and ice and snow. here we mask these pixels out
sumlandCover = pastureOrg.add(rangelandOrg).add(croplandOrg).add(urbanOrg).add(freeland);
restorationMap = potentialCoverAdjusted.subtract(presentForestCover).mask(effectivePotentialMask).unmask();

# sum all these scaled layersv
scaledSum = pasture.add(rangeland).add(cropland).add(urban).add(freeland);
potentialCoverFinal = restorationMap.add(presentForestCover);
# allocate the potential equally to each layer
freelandPotentialCover = freeland.divide(scaledSum).multiply(restorationMap).unmask();
rangelandPotentialCover = rangeland.divide(scaledSum).multiply(restorationMap).unmask();
pasturePotentialCover = pasture.divide(scaledSum).multiply(restorationMap).unmask();
croplandPotentialCover = cropland.divide(scaledSum).multiply(restorationMap).unmask();
urbanPotentialCover = urban.divide(scaledSum).multiply(restorationMap).unmask();
#  allocate the freeland potential in pixels with forest cover larger than 10% to conservation potential
freelandForConsevation = freelandPotentialCover.multiply(presentForestCover.gte(0.1)).unmask();
maximumPotentialCover = freelandForConsevation.add(presentForestCover);
# calucate the reall freeland outside of forest
freelandLeftMap = freelandPotentialCover.subtract(freelandForConsevation).unmask()# the left positive pixels are real freeland pixels

## 5 Partioning the biomass potential into different landuse types

In [24]:
# calculate the existing carbon, present potential carbon and absolute potential carbon in forests
absoluteImage1 = tgbPresentDensity.multiply(pixelAreaMap).multiply(presentMask).divide(1000000000).rename('Present')
absoluteImage3 = tgbPotentialDensity.multiply(pixelAreaMap).multiply(potentialCoverFinal.gt(0)).divide(1000000000).rename('AbsolutePotential')
# get the sum of the potential covers
potentialCoverSum = freelandLeftMap.add(rangelandPotentialCover).add(pasturePotentialCover).add(croplandPotentialCover).add(urbanPotentialCover)

trueRestorationPotential = absoluteImage3.subtract(absoluteImage1).multiply(1000000000)
ratioPotentialBiomassDensity = absoluteImage1.multiply(potentialCoverFinal.divide(presentForestCover))
#  get the real for conservation potential
realDensityIncreased = absoluteImage3.subtract(absoluteImage1).mask(absoluteImage3.subtract(ratioPotentialBiomassDensity).gt(0)).unmask()
realDensityNotIncreased = absoluteImage3.subtract(absoluteImage1).mask(absoluteImage3.subtract(ratioPotentialBiomassDensity).lte(0)).unmask()
trueReforestationPotential = realDensityNotIncreased.add(realDensityIncreased.multiply(ee.Image(1).subtract(presentForestCover.divide(potentialCoverFinal))))

conservationPotentialPart1 = realDensityIncreased.multiply(presentForestCover.add(freelandForConsevation).divide(potentialCoverFinal))
conservationPotentialPart2 = realDensityNotIncreased.multiply(freelandForConsevation.divide(potentialCoverFinal.subtract(presentForestCover)))

# calculate the part of the potential inside the forest cover which was allocate to conservation potential.
freelandForConservation1 = realDensityIncreased.multiply(freelandForConsevation.divide(potentialCoverFinal))
freelandForConservation = freelandForConservation1.add(conservationPotentialPart2).rename('FreeToConservation')

absoluteImage2 = conservationPotentialPart1.add(conservationPotentialPart2).add(absoluteImage1).rename('PresentPotential')

trueReforestationPotential = absoluteImage3.subtract(absoluteImage2)

absoluteImage4 = trueReforestationPotential.multiply(freelandLeftMap.divide(potentialCoverSum)).rename('FreelandPotential')
absoluteImage5 = trueReforestationPotential.multiply(rangelandPotentialCover.divide(potentialCoverSum)).rename('RangelandPotential')
absoluteImage6 = trueReforestationPotential.multiply(pasturePotentialCover.divide(potentialCoverSum)).rename('PasturePotential')
absoluteImage7 = trueReforestationPotential.multiply(croplandPotentialCover.divide(potentialCoverSum)).rename('CroplandPotential')
absoluteImage8 = trueReforestationPotential.multiply(urbanPotentialCover.divide(potentialCoverSum)).rename('UrbanPotential')

## Calculate the potential numbers and write into local folder

In [25]:
# Stack the absolute biomass layers into an Image.
absPotentialImage = absoluteImage1.addBands(absoluteImage2).addBands(absoluteImage3).addBands(absoluteImage4).addBands(absoluteImage5).addBands(absoluteImage6).addBands(absoluteImage7).addBands(absoluteImage8).addBands(freelandForConservation)
# define the function to do the biome level statistics which could be applied by map      
def biomeLevelStat(biome):
    biomeMask = biomeLayer.eq(ee.Number(biome))
    masked_img = absPotentialImage.mask(biomeMask)
    output = masked_img.reduceRegion(reducer= ee.Reducer.sum(),
                                     geometry= unboundedGeo,
                                     crs='EPSG:4326',
                                     crsTransform=[0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                     maxPixels= 1e13)
    return output#.getInfo().get('Present')


biomeList = ee.List([1,2,3,4,5,6,7,8,9,10,11,12,13,14])
statisticTable = biomeList.map(biomeLevelStat).getInfo()
# transform into data frame
outputTable = pd.DataFrame(statisticTable,columns =['Present','PresentPotential','AbsolutePotential','FreelandPotential','RangelandPotential','PasturePotential','CroplandPotential','UrbanPotential','FreeToConservation'])#.round(1)
outputTable.loc['sum'] = outputTable.sum()
outputTable.to_csv('Data/BiomeLevelStatistics/StatisticsForModels/GS2_MaxScaler_Biome_Level_Statistics.csv',header=True,mode='w+')
# display the output of the carbon partitioning
print(colored('The biomass partition results in biome: \n', 'blue', attrs=['bold']))
outputTable.head(15)

The biomass partition results in biome: 



,Present,PresentPotential,AbsolutePotential,FreelandPotential,RangelandPotential,PasturePotential,CroplandPotential,UrbanPotential,FreeToConservation
0,211.043662,248.394540,270.471621,4.155994,0.067065,8.075043,9.525401,0.253578,7.322728
1,5.942344,9.858025,15.618636,1.394694,0.013297,1.619896,2.702066,0.030659,0.836807
2,2.288131,4.016641,4.842772,0.273296,0.009057,0.299324,0.241617,0.002837,0.422755
3,32.903483,43.672855,56.242709,2.823930,0.015107,3.885804,5.485138,0.359875,3.143632
4,14.902299,20.231533,22.150568,0.842882,0.020226,0.578572,0.437633,0.039722,1.687036
5,21.211961,23.442787,24.640003,0.990843,0.000541,0.101987,0.097173,0.006672,1.472756
6,32.256835,58.525428,82.385103,3.993100,7.782839,7.269196,4.760556,0.053984,4.879850
7,2.498406,3.311429,11.142583,1.369962,2.227424,1.103531,3.079988,0.050249,0.169781
8,1.159241,2.024961,2.841409,0.202453,0.331054,0.178964,0.097249,0.006727,0.146960
9,2.476383,3.981194,6.103692,0.482367,0.682303,0.464372,0.489195,0.004261,0.253286


In [26]:
# If you got the error 'EEException: Too many concurrent aggregations.', please re-run this chunck of code again.

In [33]:
deadWoodLitterRatio = ee.Image("users/leonidmoore/ForestBiomass/DeadWoodLitter/DeadWood_Litter_Ratio_Map").unmask()

# Stack the absolute biomass layers into an Image.
absPotentialImage = absoluteImage1.addBands(absoluteImage2).addBands(absoluteImage3).addBands(absoluteImage4).addBands(absoluteImage5).addBands(absoluteImage6).addBands(absoluteImage7).addBands(absoluteImage8).addBands(freelandForConservation)
# define the function to do the biome level statistics which could be applied by map      
def biomeLevelStat(biome):
    biomeMask = biomeLayer.eq(ee.Number(biome))
    masked_img = absPotentialImage.mask(biomeMask).multiply(deadWoodLitterRatio)
    output = masked_img.reduceRegion(reducer= ee.Reducer.sum(),
                                     geometry= unboundedGeo,
                                     crs='EPSG:4326',
                                     crsTransform=[0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                     maxPixels= 1e13)
    return output#.getInfo().get('Present')


biomeList = ee.List([1,2,3,4,5,6,7,8,9,10,11,12,13,14])
statisticTable = biomeList.map(biomeLevelStat).getInfo()
# transform into data frame
outputTable = pd.DataFrame(statisticTable,columns =['Present','PresentPotential','AbsolutePotential','FreelandPotential','RangelandPotential','PasturePotential','CroplandPotential','UrbanPotential','FreeToConservation'])#.round(1)
outputTable.loc['sum'] = outputTable.sum()
outputTable.to_csv('Data/BiomeLevelStatistics/StatisticsForModels/GS2_MaxScaler_Biome_Level_Statistics_with_Litter.csv',header=True,mode='w+')
# display the output of the carbon partitioning
print(colored('The biomass partition results in biome: \n', 'blue', attrs=['bold']))
outputTable.head(15)

The biomass partition results in biome: 



,Present,PresentPotential,AbsolutePotential,FreelandPotential,RangelandPotential,PasturePotential,CroplandPotential,UrbanPotential,FreeToConservation
0,257.481342,303.052985,329.989068,5.070685,0.081946,9.852243,11.621844,0.309365,8.934433
1,7.249642,12.026746,19.054706,1.701523,0.016267,1.976272,3.296495,0.037404,1.020897
2,2.792059,4.901466,5.909546,0.333510,0.011038,0.365172,0.294897,0.003462,0.515987
3,43.758419,58.078876,74.794673,3.754995,0.020064,5.167687,7.294474,0.478577,4.180135
4,19.819178,26.905895,29.456870,1.120370,0.026766,0.769284,0.581750,0.052804,2.243285
5,38.159755,42.172758,44.325275,1.782425,0.000908,0.183184,0.174017,0.011983,2.649396
6,39.353621,71.401415,100.510564,4.871604,9.495258,8.868566,5.807859,0.065862,5.953419
7,3.323223,4.404537,14.819473,1.821957,2.962152,1.467629,4.096370,0.066828,0.225835
8,1.414526,2.470780,3.466972,0.247027,0.403891,0.218366,0.118694,0.008214,0.179319
9,3.285372,5.284496,8.105327,0.641006,0.906928,0.617083,0.650150,0.005664,0.336450


In [ ]:
# If you got the error 'EEException: Too many concurrent aggregations.', please re-run this chunck of code again.